# 🌐 CRS Data Loader – Five-Phase Workflow

## 📘 Overview
This notebook follows a streamlined **five-phase MDMS data upload workflow**, required for onboarding any new State/ULB into CRS and Common Services.

---

# 🟦 PHASE 1: Tenant + Branding Setup
- Upload **Tenant + Branding + Basic Admin structure**
- Only **4 mandatory tenant fields**
- Automatically generates:
  - Tenant information  
  - Branding configuration  
  - Basic localization keys  

---

# 🟩 PHASE 2: Boundary Setup
- Upload complete **boundary hierarchy**
- Upload **boundary tree** (district → region → ward → block…)
- Supports custom boundary levels (Level 1–5)
- Automatically maps boundaries to the tenant

---

# 🟪 PHASE 3: Common & Complaint Master
Configure complete CRS functional data:
- Departments  
- Designations  
- Complaint Types  
- Complaint Sub-Types  
- SLA Hours  
- Localization entries  

Automatically generates:
- Department codes  
- Designation codes  
- Complaint type IDs  
- Localization message codes


---

# 🟦 PHASE 4: Employee Bulk Creation 
Bulk create employees for your tenant:
- **Dynamic template generation** - Fetches departments, designations, roles from MDMS
- **Name-based Excel** - Use names instead of codes (auto-converted)
- **Auto role validation** - Missing roles automatically created in MDMS
- **Status tracking** - Color-coded results (🟢 SUCCESS, 🟡 EXISTS, 🔴 FAILED)
- **README - Roles sheet** - Copy-paste ready role names

Automatically handles:
- Employee code generation
- Role validation & creation
- Jurisdiction setup
- User account creation
- Error tracking with Excel updates

---

# 🔵 PHASE 5: Workflow Configuration
Manage complaint workflow configurations:
- **Download existing workflow** - Get current PGR workflow as JSON
- **Edit workflow** - Customize states, actions, roles, and SLA
- **Auto create/update** - System detects if workflow exists

Workflow elements:
- States (PENDING, ASSIGNED, RESOLVED, REJECTED, etc.)
- Actions (ASSIGN, RESOLVE, REJECT, REOPEN, etc.)
- Role-based access control
- SLA timeouts for each state

---

# ⚠️ IMPORTANT: Run Cells in Correct Order

To avoid errors, always run the notebook **top → bottom**:

1. **Initialize Global Variables** (Cell 2)  
2. **Import Dependencies** (Cell 3)  
3. **Download Templates** (Cell 4)  
4. **Proceed to Phase 1 → Phase 2 → Phase 3 → Phase 4 → Phase 5**

❌ Do **NOT** skip cells  
❌ Do **NOT** jump directly to later phases  
✔ Always complete phases sequentially



# 📝 Prerequisites

- State tenant ID (e.g., `pg`, `pb`, `mh`)
- Templates stored inside the `/templates` folder
- Phase 1-3 completed before Phase 4 (departments, designations, boundaries must exist)
- Phase 1-4 completed before Phase 5 (roles and departments should exist for workflow)

---

## 🔄 How to Use This Notebook

### Running Cells
- **Option 1:** Click on a cell and press `Shift + Enter`
- **Option 2:** Click the ▶️ Run button in the toolbar
- **Option 3:** Use menu: `Cell` → `Run Cells`

### Cell Types
- **Markdown cells** (like this one) - Instructions and documentation
- **Code cells** - Python code to execute

### Status Indicators
- `[*]` - Cell is running
- `[1]` - Cell completed (number shows execution order)
- No bracket - Cell not yet run

---



## Install Dependencies

This cell automatically installs all required Python packages:
- **pandas, openpyxl** - Excel file handling
- **requests** - API communication
- **ipywidgets** - Interactive UI components
- **jupyter** - Notebook support
- **python-dotenv** - Configuration management

**⏱️ Time:** Installation may take 1-2 minutes on first run.

**🔄 Action:** Run this cell once at the start of your session.

---

In [ ]:
# ============================================================================
# 📦 AUTOMATIC DEPENDENCY INSTALLATION
# ============================================================================
# This cell automatically installs all required packages for cross-platform compatibility
# Supports: Windows, macOS (Intel & Apple Silicon), Linux (x86 & ARM)

import sys
import subprocess
import platform
import os

def get_platform_name():
    """Get user-friendly platform name"""
    system = platform.system()
    machine = platform.machine().lower()
    
    if system == "Darwin":
        if machine in ['arm64', 'aarch64']:
            return "macOS (Apple Silicon)"
        else:
            return "macOS (Intel)"
    elif system == "Windows":
        return "Windows"
    elif system == "Linux":
        if machine in ['arm64', 'aarch64']:
            return "Linux (ARM)"
        else:
            return "Linux"
    else:
        return f"{system} ({machine})"

def install_dependencies():
    """Install all required dependencies with cross-platform support"""
    
    print("=" * 70)
    print("🔧 CRS Data Loader - Dependency Installation")
    print("=" * 70)
    print(f"📍 Platform: {get_platform_name()}")
    print(f"🐍 Python: {sys.version.split()[0]}")
    print("=" * 70)
    
    # Check if requirements.txt exists (check parent dirs)
    req_file = 'requirements.txt'
    if not os.path.exists(req_file):
        req_file = '../requirements.txt'
    
    if os.path.exists(req_file):
        print(f"✅ Found requirements.txt - Installing from file...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", req_file, "--upgrade"])
            print("\n✅ Dependencies installed from requirements.txt")
        except subprocess.CalledProcessError:
            print(f"\n⚠️  Some packages failed, trying fallback...")
            install_fallback()
    else:
        print("⚠️  requirements.txt not found - Using fallback installation...")
        install_fallback()
    
    # Enable Jupyter widgets with enhanced Linux support
    print("\n🔧 Enabling Jupyter widgets...")
    widget_success = False
    
    # Step 1: Install widget packages
    try:
        print("   Installing widget packages...")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "ipywidgets", "jupyterlab_widgets", "--quiet"],
            timeout=30
        )
        widget_success = True
    except Exception:
        pass
    
    # Step 2: Try JupyterLab extension
    try:
        print("   Building JupyterLab extensions...")
        subprocess.check_call(
            [sys.executable, "-m", "jupyter", "lab", "build", "--minimize=False"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            timeout=90
        )
        print("   ✅ JupyterLab widgets installed")
        widget_success = True
    except Exception:
        print("   ⚠️  JupyterLab build skipped (may not be needed)")
    
    # Step 3: Try classic notebook extension
    try:
        subprocess.check_call(
            [sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            timeout=30
        )
        print("   ✅ Classic notebook widgets enabled")
        widget_success = True
    except Exception:
        pass
    
    if widget_success:
        print("\n✅ Jupyter widgets configured")
    
    print("\n" + "=" * 70)
    print("✅ INSTALLATION COMPLETE!")
    print("=" * 70)
    
    # Show restart message if widgets were configured
    if widget_success and platform.system() == "Linux":
        print("💡 On Linux: If widgets don't appear, please:")
        print("   1. Stop this notebook (Ctrl+C in terminal)")
        print("   2. Restart: jupyter lab (or jupyter notebook)")
        print("   3. Refresh your browser (Ctrl+Shift+R)")
        print("=" * 70)
    
    print("📝 You can now run the next cells")
    print("=" * 70)

def install_fallback():
    """Fallback installation if requirements.txt is missing"""
    
    core_packages = [
        "pandas>=2.0.0",
        "openpyxl>=3.1.0",
        "xlsxwriter>=3.1.0",
        "requests>=2.31.0",
        "python-dotenv>=1.0.0",
        "notebook>=7.0.0",
        "jupyterlab>=4.0.0",
        "ipykernel>=6.25.0",
        "ipywidgets>=8.1.0",
        "jupyterlab_widgets>=3.0.0",
        "tqdm>=4.65.0",
        "plotly>=5.17.0",
        "sqlalchemy>=2.0.0",
        "typing-extensions>=4.7.0"
    ]
    
    print("\n📦 Installing core packages...")
    for package in core_packages:
        try:
            pkg_name = package.split('>=')[0]
            print(f"   Installing {pkg_name}...", end=" ")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", package, "--quiet"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print("✅")
        except subprocess.CalledProcessError:
            print("⚠️")
    
    print("\n📦 Installing database adapter...")
    machine = platform.machine().lower()
    if machine not in ['aarch64', 'arm64']:
        try:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "psycopg2-binary>=2.9.0", "--quiet"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print("   ✅ psycopg2-binary installed")
        except:
            print("   ⚠️  psycopg2-binary skipped (optional)")

# Run installation
try:
    install_dependencies()
except Exception as e:
    print(f"\n❌ Installation failed: {str(e)}")
    print("\n💡 Manual installation - Run in terminal:")
    print("   pip install -r requirements.txt")
    print("   jupyter lab build")


In [ ]:
import os
from IPython.display import display, HTML

templates_path = "templates"

template_files = [
    ("Tenant And Branding Master.xlsx", "PHASE 1: Tenant Info (4 fields) + Branding"),
    ("Common and Complaint Master.xlsx", "PHASE 3: Departments, Designations & Complaint Types"),
]

html_links = []
for i, (filename, description) in enumerate(template_files, 1):
    filepath = os.path.join(templates_path, filename)
    if os.path.exists(filepath):
        html_links.append(
            f'''
            <div style="margin: 8px 0;">
                <b>{i}.</b>
                <a href="{filepath}" download="{filename}"
                   style="font-size: 14px; color: #0066cc; text-decoration: none;">
                    {filename}
                </a>
                <span style="color: #666;"> - {description}</span>
            </div>
            '''
        )
    else:
        html_links.append(
            f'''
            <div style="margin: 8px 0;">
                <b>{i}.</b> {filename}
                <span style="color: red;">(Not found)</span>
            </div>
            '''
        )

html_content = f"""
<div style="border: 1px solid #ddd; padding: 15px; border-radius: 6px; background-color: #f9f9f9;">
    <h3 style="margin-top: 0;">📁 Download MDMS  Templates</h3>

    {''.join(html_links)}

    <hr style="margin: 18px 0;">

    <p><b>🧭 4-Phase Workflow:</b></p>
    <ol>
        <li><b>PHASE 1:</b> Tenant + Branding </li>
        <li><b>PHASE 2:</b> Boundary Master (Auto Generated based on userInput)</li>
        <li><b>PHASE 3:</b> Departments, Designations & Complaint Types</li>
            <li><b>PHASE 4:</b> Employee Master (Auto Generated based on userInput)</li>
    </ol>

  
</div>
"""

display(HTML(html_content))


## Initialize System

This cell prepares the environment by:
- ✅ Importing required Python modules
- ✅ Clearing previous upload files from `upload/` folder
- ✅ Reloading latest code changes from `unified_loader.py`
- ✅ Initializing global variables

**🔄 Action Required:** Run this cell once at the start.

**⚠️ Important:** After running this cell, proceed to authentication (Phase 0).

---

In [ ]:
# Import required packages
import pandas as pd
import json
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans
import shutil
import os
import sys
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Initialize Global Variables
CONFIG = {}
CONFIG_SET = False
TENANT_FILE = None
COMMON_MASTER_FILE = None
UPLOADED_TENANTS = []
SELECTED_TENANT = None
UPLOADER = None  # Global authenticated uploader instance

# Force reload modules to get latest changes
if 'unified_loader' in sys.modules:
    del sys.modules['unified_loader']
if 'mdms_validator' in sys.modules:
    del sys.modules['mdms_validator']
if 'excel_validator' in sys.modules:
    del sys.modules['excel_validator']

# Clear upload folder
upload_dir = 'upload'
if os.path.exists(upload_dir):
    for filename in os.listdir(upload_dir):
        file_path = os.path.join(upload_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f"✅ Cleared all files from {upload_dir}/")

print("✅ Packages loaded successfully!")
print("✅ Modules reloaded with latest changes!")
print()
print("="*70)
print("  ⚙️ AUTHENTICATION REQUIRED")
print("="*70)
print("Please proceed to the next cell to authenticate with the gateway.")

---

# 🔐 PHASE 0: AUTHENTICATION

## 📋 Overview
Before you can upload any data, you must authenticate with the DIGIT gateway using OAuth2.

---

## 🎯 What You'll Do:
1. Enter your **Gateway URL** (domain)
2. Enter **Admin Credentials** (username and password)
3. Select **User Type** (EMPLOYEE)
4. Enter **Root Tenant ID** (e.g., `pg`, `pb`, `statea`)
5. Click **"🔐 Authenticate"** button

---

## ✅ What Gets Created:
- OAuth2 access token (valid for session)
- User session with assigned roles
- Authenticated API client for all subsequent operations

---

## ⚠️ Important:
- **Required Role:** MDMS_ADMIN, SUPERUSER, or LOC_ADMIN
- **Security:** Credentials are used only for this session and not stored
- **Token Expiry:** Token expires when you close the notebook

---

## 🔧 Troubleshooting:
- **Authentication Failed:** Check username, password, and tenant ID
- **Invalid Role:** Contact your system administrator to assign required roles
- **Network Error:** Verify gateway URL and internet connection

---

In [ ]:
print("="*70)
print("        🔐 GATEWAY AUTHENTICATION")
print("="*70)


# Authentication widgets
base_url_input = widgets.Text(
    value="",
    description='🌐 Domain URL:',
    placeholder='e.g., https://unified-dev.digit.org',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

username_input = widgets.Text(
    value="",
    description='👤 Username:',
    placeholder='e.g., SUPERADMIN,CITYADMIN',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

password_input = widgets.Password(
    value="",
    description='🔑 Password:',
    placeholder='Enter password',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

user_type_input = widgets.Dropdown(
    options=['EMPLOYEE'],
    value="EMPLOYEE",
    description='👔 User Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tenant_id_input = widgets.Text(
    value="",
    description='🏛️ Tenant ID:',
    placeholder='statea,statea.citya',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

auth_button = widgets.Button(
    description='🔐 Authenticate',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

auth_output = widgets.Output()

def on_authenticate(b):
    global UPLOADER, CONFIG
    
    with auth_output:
        clear_output()
        
        # Validate inputs
        if not base_url_input.value.strip():
            print("❌ Domain URL is required")
            return
        
        if not username_input.value.strip():
            print("❌ Username is required")
            return
        
        if not password_input.value.strip():
            print("❌ Password is required")
            return
        
        if not tenant_id_input.value.strip():
            print("❌ Tenant ID is required")
            return
        
        print("🔄 Authenticating with gateway...")
        print(f"   Gateway: {base_url_input.value}")
        print(f"   Username: {username_input.value}")
        print(f"   User Type: {user_type_input.value}")
        print(f"   Tenant ID: {tenant_id_input.value}")
        print()
        
        try:
            # Create APIUploader with authentication
            UPLOADER = APIUploader(
                base_url=base_url_input.value.strip(),
                username=username_input.value.strip(),
                password=password_input.value.strip(),
                user_type=user_type_input.value,
                tenant_id=tenant_id_input.value.strip()
            )
            
            if UPLOADER.authenticated:
                # Save config
                CONFIG = {
                    'base_url': base_url_input.value.strip(),
                    'tenant_id': tenant_id_input.value.strip()
                }
                CONFIG_SET = True
                
                print()
                print("="*70)
                print("  ✅ AUTHENTICATION SUCCESSFUL!")
                print("="*70)
                print(f"\n👤 Logged in as: {UPLOADER.user_info.get('userName', 'Unknown')}")
                print(f"📧 Email: {UPLOADER.user_info.get('emailId', 'N/A')}")
                print(f"🏛️ Tenant: {UPLOADER.user_info.get('tenantId', 'N/A')}")
                print(f"\n🎭 Roles:")
                for role in UPLOADER.user_info.get('roles', []):
                    print(f"   • {role.get('name', 'Unknown')} ({role.get('code', 'N/A')})")
                print()
                print("="*70)
                print("  ➡️ Proceed to PHASE 1: Tenant Setup")
                print("="*70)
            else:
                print("❌ Authentication failed. Please check your credentials.")
        
        except Exception as e:
            print(f"❌ Error during authentication: {str(e)}")

auth_button.on_click(on_authenticate)

# Display UI
auth_ui = widgets.VBox([
    widgets.HTML("<a id='login-step'></a>"),
    widgets.HTML("<h3>🔐 Step 0: Authentication</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Authenticate with the eGov gateway to access all services</i></p>"),
    widgets.HTML("<br>"),
    base_url_input,
    widgets.HTML("<p style='font-size: 11px; color: #888;'><i>💡 Gateway URL - same for all services</i></p>"),
    widgets.HTML("<br>"),
    username_input,
    password_input,
    user_type_input,
    tenant_id_input,
    widgets.HTML("<p style='font-size: 11px; color: #ff6600;'><i>⚠️ Credentials are used only for this session and not stored</i></p>"),
    widgets.HTML("<br>"),
    auth_button,
    auth_output
])

display(auth_ui)

# Tenant Configuration

Choose whether to use an existing tenant or create a new one:

**Option A: Use Existing Tenant**
- Select from dropdown of tenants already in MDMS
- Useful for adding data to existing tenants
- Click **✅ Proceed** to continue with the selected tenant

**Option B: Create New Tenant**
- Enter new tenant code (e.g., `statea`, `stateb`)
- System will create default tenant structure
- Required for new state/ULB onboarding
- It automatically logs in with the tenant using the SUPERADMIN credentials provided during authentication.
- Click **✅ Proceed** to create the tenant

## ⚠️ IMPORTANT: After Creating New Tenant

When you create a **new tenant**, a **🔐 Login Again** button will appear after successful creation.

**Why you need to login again:**
- You authenticated with the parent tenant (e.g., `pg`)
- You just created a new tenant (e.g., `statea`)
- To manage this new tenant's data, you must re-authenticate using the **new tenant's admin credentials**

**Steps after tenant creation:**
1. ✅ Wait for "Tenant created successfully!" message
2. It automatically logs in with the tenant using the previous credentials provided during authentication.
3. 🔐 Click the **Login Again** button if required
4. ▶️ Continue with Phase 1 to upload tenant master data

**💡 Note:** You can upload data to any tenant - it doesn't have to be the one you authenticated with.

---

In [ ]:
import ipywidgets as widgets
import sys
from IPython.display import display, clear_output, Javascript, HTML


# Initialize uploader
uploader = UPLOADER
if uploader is None:
    display(widgets.HTML(
        "<div style='background:#fff4cd;padding:10px;border-radius:5px;'>"
        "<b>⚠ Authenticate in order to proceed.</b></div>"
    ))
    raise SystemExit
else:
    # hardcoded
    tenants = uploader.search_mdms_data(schema_code='tenant.tenants', tenant=CONFIG.get('tenant_id'))
    
    # Fetch existing tenants
tenant_codes = [t["code"] for t in tenants] if tenants else []

    # Title
title = widgets.HTML("<h3 style='color:#444;'>⚙️ Tenant Configuration</h3>")
subtitle = widgets.HTML("<h4 style='color:#FF0000;margin-bottom:10px;'>Once the new tenant is created, please log in again using the new root tenant admin credentials.</h4>")

# Info box
if tenants:
    tenant_list_html = "<br>".join([f"• <b>{t['code']}</b> - {t['name']}" for t in tenants])
    info_box = widgets.HTML(
        f"<div style='background:#e7f3ff;padding:10px;border-radius:5px;'>"
        f"<b>✔ Found {len(tenants)} existing tenant(s) in MDMS:</b><br>{tenant_list_html}</div>"
    )
else:
    info_box = widgets.HTML(
        "<div style='background:#fff3cd;padding:10px;border-radius:5px;'>"
        "<b>⚠ No existing tenants found in MDMS.</b><br>"
        "<i>You can only create a new tenant.</i></div>"
    )

# YES/NO question
question = widgets.HTML("<p><b>Do you want to use an existing tenant?</b></p>")

# YES/NO buttons - conditional based on whether tenants exist
if tenants:
    # Tenants exist: show both options, default to YES
    yes_no_buttons = widgets.ToggleButtons(
        options=[('YES - Use Existing', 'yes'), ('NO - Create New', 'no')],
        value='yes'  # Pre-select YES when tenants exist
    )
else:
    # No tenants: only show NO option (or disable YES)
    yes_no_buttons = widgets.ToggleButtons(
        options=[('NO - Create New', 'no')],
        value='no'  # Pre-select NO when no tenants exist
    )

# Existing tenant dropdown - visible by default if tenants exist
existing_dropdown = widgets.Dropdown(
    options=tenant_codes,
    description='Select Tenant:',
    style={'description_width': '120px'},
    layout=widgets.Layout(visibility='visible' if tenants else 'hidden')
)

# New tenant UI (each field hidden individually)
new_label = widgets.HTML(
    "<p style='color:#666;font-size:12px;'><i>💡 Fill in details for the new tenant</i></p>",
    layout=widgets.Layout(visibility='hidden' if tenants else 'visible')
)

new_tenant_code = widgets.Text(
    placeholder='e.g., statea, stateb',
    description='Tenant Code:',
    style={'description_width': '120px'},
    layout=widgets.Layout(visibility='hidden' if tenants else 'visible')
)
login_again_button = widgets.HTML(
    """
    <div style="margin-top:12px;">
        <a href="#login-step" style="text-decoration:none;">
            <button style="
                padding:8px 16px;
                font-size:14px;
                background:#f0ad4e;
                border:none;
                border-radius:4px;
                cursor:pointer;">
                🔐 Login Again
            </button>
        </a>
    </div>
    """)
login_again_button.layout.visibility = 'hidden'
# Proceed button - visible by default
proceed_button = widgets.Button(
    description="✅ Proceed",
    button_style='success',
    icon='check',
    layout=widgets.Layout(visibility='visible')
)


output = widgets.Output()

# ----- VISIBILITY TOGGLE -----
def on_yes_no_change(change):
    choice = change['new']
    if choice == 'yes':
        existing_dropdown.layout.visibility = 'visible'
        # Hide new tenant input fields
        for w in [
            new_label,
            new_tenant_code
        ]:
            w.layout.visibility = 'hidden'
        proceed_button.layout.visibility = 'visible'
    elif choice == 'no':
        existing_dropdown.layout.visibility = 'hidden'
        # Show new tenant input fields
        for w in [
            new_label,
            new_tenant_code
        ]:
            w.layout.visibility = 'visible'
        proceed_button.layout.visibility = 'visible'
       

yes_no_buttons.observe(on_yes_no_change, names='value')

# ----- PROCEED BUTTON LOGIC -----
def on_proceed_clicked(b):
    with output:
        clear_output()
        # Existing tenant selected
        if yes_no_buttons.value == 'yes':
            selected = existing_dropdown.value
            print(f"✅ Proceeding with existing tenant: {selected}")
            return
        
        # New tenant flow
        tenant_code = new_tenant_code.value.strip()
        
        # Validate no dots or special characters
        import re
        if '.' in tenant_code or not re.match(r'^[a-zA-Z0-9_]+$', tenant_code):
            print("❌ Tenant code cannot contain dots (.) or special characters")
            print("   Use simple names like: statea, stateb, statec")
            return
        
        # Validation
        if not tenant_code:
            print("❌ Tenant code is required")
            return
        
        print(f"🔄 Creating new tenant: {tenant_code}...\n")
        print("⏳ Please wait, this operation may take 5-10 minutes...")
        print("")
        sys.stdout.flush()
        
        # ---- CORRECT API CALL ----
        result = uploader.setup_default_data(
            targetTenantId=tenant_code,
            module="tenant",
            schemaCodes=[
              "tenant.citymodule",
              "tenant.tenants"
            ],
            onlySchemas=False
        )
        
        if result.get("success"):
            print("✅ Tenant created successfully!")
            login_again_button.layout.visibility = 'visible'
            # Update CONFIG with new tenant
            CONFIG['tenant_id'] = tenant_code
            print(f"Please log in again if required using the login again button.")
        else:
            print("❌ Failed to create tenant")
            print(result)

proceed_button.on_click(on_proceed_clicked)

# UI Layout
ui = widgets.VBox([
    title,
    subtitle,
    info_box,
    question,
    yes_no_buttons,
    existing_dropdown,
    new_label,
    new_tenant_code,
    proceed_button,
    output,
    login_again_button 
])

display(ui)






---

# 🟦 PHASE 1: TENANT/STATE SETUP

## 📋 Overview
In Phase 1, you will set up the foundational tenant and branding configuration for your state/ULB.

**🔐 Authentication Level:** Use **STATE-level user** (e.g., authenticated with `statea`)

## 🎯 What You'll Do:
1. **Enter Root/State Tenant ID** (e.g., `pg` for Punjab, `pb` for Punjab, `mh` for Maharashtra)
2. **Upload Tenant Master Excel** with state and city information
3. **Upload tenant data** - Creates tenant records in MDMS
4. **Upload state branding** - Logo, colors, theme configuration

## 📄 Template Required:
- **Tenant And Branding Master.xlsx**

## ✅ What Gets Created:
- Tenant information in MDMS
- City-level tenant records (e.g., `pg.citya`)
- State Info uploaded to both **root level** and **state level**
- Branding configuration (logo, theme, colors) uploaded to both **root level** and **state level**
- Basic localization keys
- Admin boundary structure

## ⚠️ Important:
- **Use STATE-level authentication** (e.g., `statea`) for this phase
- Tenant ID must be lowercase (e.g., `pg`, not `PG`)
- Complete Phase 1 before moving to Phase 2

In [ ]:
# Configuration Setup and Tenant Master Upload
# Make sure you've run the "Initialize Global Variables" cell first!

print("="*70)
print("        ⚙️  PHASE 1: CONFIGURATION & UPLOAD TENANT MASTER")
print("="*70)
print()

# Configuration widgets
state_tenant_id = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    description='Root Tenant ID:',
    placeholder='e.g., pg, pb, up',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
tenant_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined button
upload_btn = widgets.Button(
    description='✅ Save Config & Upload Tenant Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

output = widgets.Output()

def on_upload(b):
    global CONFIG, CONFIG_SET, TENANT_FILE, UPLOADED_TENANTS
    global result_tenants, result_branding
    
    with output:
        clear_output()
        
        # Validate configuration
        if not state_tenant_id.value.strip():
            print("❌ Root Tenant ID required")
            return
        
        # Validate file selection
        if not tenant_file_upload.value:
            print("❌ Please select Tenant Master Excel file")
            return
        
        # Validate authentication
        if not UPLOADER or not UPLOADER.authenticated:
            print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
            return
        
        # Save configuration
        CONFIG = {
            'base_url': 'https://unified-qa.digit.org',
            'tenant_id': state_tenant_id.value.strip().lower()
        }
        CONFIG_SET = True
        
        print("="*70)
        print("  ✅ CONFIGURATION SAVED!")
        print("="*70)
        print(f"\n📊 Configuration:")
        print(f"   • Root Tenant: {CONFIG.get('tenant_id')}")
        print("="*70)
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = tenant_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Tenant_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("\n" + "="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        validation_passed = False
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator(
                base_url=UPLOADER.base_url,
                auth_token=UPLOADER.auth_token,
                user_info=UPLOADER.user_info
            )
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=CONFIG.get('tenant_id'),
                schema_code='tenant.masterschemavalidation'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                validation_passed = True
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                TENANT_FILE = None
                CONFIG_SET = False
                return
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            validation_passed = True
        
        if not validation_passed:
            return
        
        TENANT_FILE = upload_path
        
        print("\n" + "="*70)
        print("  ✅ FILE UPLOADED SUCCESSFULLY")
        print("="*70)
        print(f"\n📄 File: {upload_path}")
        
        # ============================================================
        # INTEGRATED API CALLS - Start processing immediately
        # ============================================================
        
        print("\n" + "="*70)
        print("[PHASE 1] LOADING TENANT DATA")
        print("="*70)
        
        try:
            # Initialize reader
            reader = UnifiedExcelReader(TENANT_FILE)
            uploader = UPLOADER
            
            # Read tenant data using NEW format method
            tenants_data, tenants_localization = reader.read_tenant_info()
            
            print(f"\n[INFO] Loaded {len(tenants_data)} tenant(s) from Excel")
            for tenant in tenants_data:
                print(f"   - {tenant['code']}: {tenant['name']} ({tenant['type']})")
                UPLOADED_TENANTS.append(tenant['code'])
            
            print(f"\n[INFO] Auto-generated {len(tenants_localization)} localization entries")
            
            # Upload Tenants
            result_tenants = uploader.create_mdms_data(
                schema_code='tenant.tenants',
                data_list=clean_nans(tenants_data),
                tenant=CONFIG.get('tenant_id'),
                sheet_name='Tenant Info',
                excel_file=TENANT_FILE
            )
            
            # Upload Localization
            result_loc = uploader.create_localization_messages(
                localization_list=clean_nans(tenants_localization),
                tenant=CONFIG.get('tenant_id'),
                sheet_name='Tenants_Localization'
            )
            
            if result_tenants['failed'] == 0:
                print("\n✅ [SUCCESS] Tenants uploaded successfully!")
            else:
                print("\n⚠️  [WARNING] Some tenants failed. Check errors/ folder.")
            
            # ============================================================
            # PHASE 1.2: State Branding (if available)
            # ============================================================
            
            print("\n" + "="*70)
            print("[PHASE 1] LOADING STATE BRANDING DATA")
            print("="*70)
            
            # Read branding data using NEW format method
            branding_data = reader.read_tenant_branding(CONFIG.get('tenant_id'))
            
            if not branding_data:
                print("\n[INFO] No branding data found in 'Tenant Branding Deatils' sheet")
                print("   This is optional - skipping this step")
            else:
                print(f"\n[INFO] Loaded {len(branding_data)} branding record(s)")
                for brand in branding_data:
                    print(f"   - {brand['code']}")
                
                # Upload Branding to tenant.stateInfo schema
                result_branding = uploader.create_mdms_data(
                    schema_code="common-masters.StateInfo",
                    data_list=clean_nans(branding_data),
                    tenant=CONFIG.get('tenant_id'),
                    sheet_name='Tenant Branding Deatils',
                    excel_file=TENANT_FILE
                )
                
                if result_branding['failed'] == 0:
                    print("\n✅ [SUCCESS] State Branding uploaded successfully!")
                else:
                    print("\n⚠️  [WARNING] Some branding records failed. Check errors/ folder.")
            
            print("\n" + "="*70)
            print("🎉 PHASE 1 COMPLETED!")
            print("="*70)
            print("\n➡️  Proceed to PHASE 2: BOUNDARY MANAGEMENT WORKFLOW")
            print("="*70)
            
        except Exception as e:
            print(f"\n❌ ERROR during data processing: {str(e)}")
            import traceback
            print("\nFull traceback:")
            traceback.print_exc()
            print("\n⚠️  Please check the error and try again.")

upload_btn.on_click(on_upload)

# Display
combined_box = widgets.VBox([
    widgets.HTML("<h3>📋 Step 1.1: Configuration & Upload Tenant Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter your root tenant ID and upload the Tenant Master Excel file</i></p>"),
    widgets.HTML("<p style='color: #666;'><b>Example:</b> If your state is Punjab, enter <code>pg</code></p>"),
    widgets.HTML("<br>"),
    state_tenant_id,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 The Tenant Master Excel will contain cities like pg.citya, pg.cityb under this state tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Tenant Master Excel File:</b></p>"),
    tenant_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated, read, and uploaded to API automatically</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    output
])

display(combined_box)


---

# 📊 INDVIDUAL SUMMARY REPORT

## 📈 Upload Status

This report shows the complete status of all uploaded data across all phases:

**Status Indicators:**
- ✅ **Created:** Successfully uploaded new records to MDMS
- ⚠️ **Already Exists:** Duplicate records (skipped, no action needed)
- ❌ **Failed:** Records with errors (see error messages)

---

## 📥 Download Updated Excel Files

Your original Excel files have been updated with status columns:
- **_STATUS** - SUCCESS / EXISTS / FAILED
- **_STATUS_CODE** - HTTP status code (200, 400, 500, etc.)
- **_ERROR_MESSAGE** - Detailed error description for failed records

**Color Coding:**
- 🟢 **GREEN rows:** Successfully created
- 🟡 **YELLOW rows:** Already exist (duplicates)
- 🔴 **RED rows:** Failed (check _ERROR_MESSAGE)

---

## 🔄 To Retry Failed Records:
1. Download the updated Excel file below
2. Fix the errors based on _ERROR_MESSAGE column
3. Delete the status columns (_STATUS, _STATUS_CODE, _ERROR_MESSAGE)
4. Re-upload the file in the respective phase

---

In [ ]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

if 'result_tenants' in locals():
    summary_data.append({
        'module': 'Tenants',
        'created': result_tenants.get('created', 0),
        'exists': result_tenants.get('exists', 0),
        'failed': result_tenants.get('failed', 0),
        'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
        'sheet': 'Tenant Info'
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""



if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS/HRMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔒 Note:</b> Status columns are <b>protected</b> and cannot be edited. Fix data in other columns and re-upload if needed.
        </p>
    </div>
    """



html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")

---

# 🟩 PHASE 2: BOUNDARY MANAGEMENT WORKFLOW

This phase implements the complete boundary management workflow with template generation.

**🔐 Authentication Level:** **MUST re-authenticate with CITY-level user** (e.g., `statea.citya`) before uploading boundary data

## ⚠️ CRITICAL: Re-Authentication Required

**Why you need city-level authentication:**
- Boundary data must be seeded at the **city/ULB level**, not state level
- You likely authenticated with state-level credentials (e.g., `statea`)
- To upload boundaries for a city, you must use **city-level admin credentials**

**When to re-authenticate:**
- After downloading the boundary template, a **🔐 Login Again (City Level)** button will appear
- Click this button to jump back to the login section
- Login with city-level credentials (e.g., `statea.citya`)
- Then proceed to upload the boundary data

---

## 📋 Workflow Steps:

### **Option 1: Create New Hierarchy (First Time)**
1. Enter tenant ID and hierarchy type
2. Define boundary levels (City → Zone → Block → Locality)
3. Create the hierarchy definition
4. Generate and download Excel template
5. Fill template with boundary data
6. **⚠️ Click the 🔐 Login Again (City Level) button**
7. **Re-authenticate with city-level credentials** (e.g., `statea.citya`)
8. Upload and process filled template in Section 2.2

### **Option 2: Use Existing Hierarchy**
1. Search for existing hierarchies
2. Select hierarchy and generate template
3. Download and fill template
4. **⚠️ Click the 🔐 Login Again (City Level) button**
5. **Re-authenticate with city-level credentials** (e.g., `statea.citya`)
6. Upload filled template in Section 2.2

## 🎯 What You'll Create:
- Boundary hierarchy definition (levels and structure)
- Excel template with pre-defined structure
- Complete boundary tree (after filling and uploading)

## 📄 Generated Files:
- Dynamic Excel template based on hierarchy
- Boundary data in MDMS after upload

---


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from unified_loader import APIUploader
import time
import os

print("="*70)
print("   🔍 PHASE 2.1: BOUNDARY HIERARCHY & TEMPLATE GENERATION")
print("="*70)



# Global variables
BOUNDARY_TENANT = None
BOUNDARY_HIERARCHY_TYPE = None
TEMPLATE_FILESTORE_ID = None
DOWNLOADED_TEMPLATE_PATH = None
DOWNLOAD_URL = None


if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

uploader = UPLOADER

# UI Widgets
tenant_input = widgets.Text(
    value= CONFIG.get('tenant_id') or 'pg',
    description='🏛️ Tenant ID:',
    placeholder='e.g., pg, mz, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tab_selector = widgets.Tab()

# TAB 1: CREATE NEW HIERARCHY
hierarchy_type_input = widgets.Text(
    value='ADMIN',
    description='Hierarchy Type:',
    placeholder='e.g., ADMIN, ADMIN1',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

level_widgets = []
levels_container = widgets.VBox([])

add_level_btn = widgets.Button(
    description='➕ Add Level',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

create_hierarchy_btn = widgets.Button(
    description='🏗️ Create Hierarchy',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

generate_template_btn = widgets.Button(
    description='📄 Generate Template',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

download_template_btn = widgets.Button(
    description='📥 Download Template',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)


# Login again button for re-authentication reminder
boundary_login_again_button = widgets.HTML(
    """
    <div style="margin-top:12px;">
        <a href="#login-step" style="text-decoration:none;">
            <button style="
                padding:8px 16px;
                font-size:14px;
                background:#f0ad4e;
                border:none;
                border-radius:4px;
                cursor:pointer;">
                🔐 Login Again (City Level)
            </button>
        </a>
    </div>
    """)
boundary_login_again_button.layout.visibility = "visible"


create_output = widgets.Output()

# TAB 2: SEARCH EXISTING
search_btn = widgets.Button(
    description='🔍 Search Hierarchies',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

hierarchy_dropdown = widgets.Dropdown(
    options=[],
    description='Hierarchy:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%', visibility='hidden')
)

search_generate_btn = widgets.Button(
    description='📄 Generate Template',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

search_download_btn = widgets.Button(
    description='📥 Download Template',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

search_output = widgets.Output()

# Helper functions
def add_initial_level():
    global level_widgets
    level_widget = widgets.Text(
        value='',
        description='Level 1:',
        placeholder='e.g., City',
        style={'description_width': '140px'},
        layout=widgets.Layout(width='95%')
    )
    level_widgets.append(level_widget)
    levels_container.children = tuple(level_widgets)

def on_add_level_clicked(b):
    global level_widgets
    level_num = len(level_widgets) + 1
    level_widget = widgets.Text(
        value='',
        description=f'Level {level_num}:',
        placeholder=f'e.g., Zone, Block',
        style={'description_width': '140px'},
        layout=widgets.Layout(width='95%')
    )
    level_widgets.append(level_widget)
    levels_container.children = tuple(level_widgets)

add_level_btn.on_click(on_add_level_clicked)

def on_create_hierarchy_clicked(b):
    global BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE

    with create_output:
        clear_output()

        tenant = tenant_input.value.strip()
        if not tenant:
            print("❌ Please enter a tenant ID")
            return

        hierarchy_type = hierarchy_type_input.value.strip()
        if not hierarchy_type:
            print("❌ Please enter a hierarchy type")
            return

        levels = [lw.value.strip() for lw in level_widgets if lw.value.strip()]
        if len(levels) == 0:
            print("❌ Please add at least one boundary level")
            return

        BOUNDARY_TENANT = tenant
        BOUNDARY_HIERARCHY_TYPE = hierarchy_type

        print(f"🏗️ Creating boundary hierarchy...")
        print(f"   Tenant: {tenant}")
        print(f"   Hierarchy Type: {hierarchy_type}")
        print(f"   Levels: {' → '.join(levels)}")

        boundary_items = []
        prev_level = None

        for level in levels:
            boundary_items.append({
                "boundaryType": level,
                "parentBoundaryType": prev_level,
                "active": True
            })
            prev_level = level

        hierarchy_payload = {
            "tenantId": tenant,
            "hierarchyType": hierarchy_type,
            "boundaryHierarchy": boundary_items
        }

        try:
            result = uploader.create_boundary_hierarchy(hierarchy_payload)
            if result:
                print("\n➡️  Now generate the Excel template")
                generate_template_btn.layout.visibility = 'visible'
        except Exception as e:
            print(f"\n❌ Failed: {str(e)[:200]}")

create_hierarchy_btn.on_click(on_create_hierarchy_clicked)

def on_generate_template_clicked(b):
    global TEMPLATE_FILESTORE_ID

    with create_output:
        clear_output(wait=True)

        if not BOUNDARY_TENANT or not BOUNDARY_HIERARCHY_TYPE:
            print("❌ Please create hierarchy first")
            return

        print(f"📄 Generating template...")

        result = uploader.generate_boundary_template(BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE)
        if not result:
            print("❌ Generation failed")
            return

        print(f"\n⏳ Waiting for completion...")
        resource = uploader.poll_boundary_template_status(
            BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE, max_attempts=30, delay=2
        )

        if resource and resource.get('status') == 'completed':
            TEMPLATE_FILESTORE_ID = resource.get('fileStoreid')
            print(f"\n✅ Template ready!")
            download_template_btn.layout.visibility = 'visible'
        else:
            print("\n❌ Failed or timed out")

generate_template_btn.on_click(on_generate_template_clicked)


def on_download_template_clicked(b):
    global DOWNLOADED_TEMPLATE_PATH, DOWNLOAD_URL

    with create_output:
        if not TEMPLATE_FILESTORE_ID:
            print("❌ No template available")
            return

        print(f"\n📥 Downloading...")

        result = uploader.download_boundary_template(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=TEMPLATE_FILESTORE_ID,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            return_url=True
        )

        if result:
            DOWNLOADED_TEMPLATE_PATH = result['path']
            DOWNLOAD_URL = result['url']
            
            print(f"\n🎉 Downloaded!")
            print(f"\n" + "="*70)
            
            # Display clickable download link
            download_link_html = f"""
            <div style='background:#e7f3ff; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#0066cc;'>📥 Template Ready!</h3>
                <p><b>Local Path:</b> {DOWNLOADED_TEMPLATE_PATH}</p>
                <a href='{DOWNLOAD_URL}' download='boundary_template.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#007bff; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold; margin:10px 0;'>
                    ⬇️ Click Here to Download Template
                </a>
            </div>
            """
            display(HTML(download_link_html))
            
            print("📝 NEXT STEPS:")
            print("1. Click the download link above OR open file locally")
            print("2. Fill boundary data in Excel")
            print("3. Save the file")
            print("4. ⚠️  IMPORTANT: Re-authenticate with CITY-level credentials before upload")
            print("5. Proceed to Step 2.2 to upload")
            print("="*70)
            
            

download_template_btn.on_click(on_download_template_clicked)

# TAB 2 HANDLERS
def on_search_clicked(b):
    global BOUNDARY_TENANT

    with search_output:
        clear_output()

        tenant = tenant_input.value.strip()
        if not tenant:
            print("❌ Please enter tenant ID")
            return

        BOUNDARY_TENANT = tenant
        print(f"🔍 Searching hierarchies in: {tenant}")

        hierarchies = uploader.search_boundary_hierarchies(tenant)

        if hierarchies:
            hierarchy_options = [(f"{h['hierarchyType']} ({len(h.get('boundaryHierarchy', []))} levels)", h['hierarchyType']) for h in hierarchies]
            hierarchy_dropdown.options = hierarchy_options
            hierarchy_dropdown.layout.visibility = 'visible'
            search_generate_btn.layout.visibility = 'visible'
        else:
            print("\n⚠️  No hierarchies found")

search_btn.on_click(on_search_clicked)

def on_search_generate_clicked(b):
    global BOUNDARY_HIERARCHY_TYPE, TEMPLATE_FILESTORE_ID

    with search_output:
        clear_output(wait=True)

        hierarchy_type = hierarchy_dropdown.value
        if not hierarchy_type:
            print("❌ Please select hierarchy")
            return

        BOUNDARY_HIERARCHY_TYPE = hierarchy_type
        print(f"📄 Generating template for {hierarchy_type}...")

        result = uploader.generate_boundary_template(BOUNDARY_TENANT, hierarchy_type)
        if not result:
            return

        resource = uploader.poll_boundary_template_status(
            BOUNDARY_TENANT, hierarchy_type, max_attempts=30, delay=2
        )

        if resource and resource.get('status') == 'completed':
            TEMPLATE_FILESTORE_ID = resource.get('fileStoreid')
            print(f"\n✅ Ready!")
            search_download_btn.layout.visibility = 'visible'

search_generate_btn.on_click(on_search_generate_clicked)

def on_search_download_clicked(b):
    global DOWNLOADED_TEMPLATE_PATH, DOWNLOAD_URL

    with search_output:
        if not TEMPLATE_FILESTORE_ID:
            return

        result = uploader.download_boundary_template(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=TEMPLATE_FILESTORE_ID,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            return_url=True
        )

        if result:
            DOWNLOADED_TEMPLATE_PATH = result['path']
            DOWNLOAD_URL = result['url']
            
            # Display clickable download link
            download_link_html = f"""
            <div style='background:#e7f3ff; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#0066cc;'>📥 Template Ready!</h3>
                <a href='{DOWNLOAD_URL}' download='boundary_template.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#007bff; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold;'>
                    ⬇️ Click to Download Template
                </a>
            </div>
            """
            display(HTML(download_link_html))
            print(f"\n✅ Downloaded! Fill template and proceed to Step 2.2")
            print("⚠️  IMPORTANT: Re-authenticate with CITY-level credentials before upload")

search_download_btn.on_click(on_search_download_clicked)

# Build Tab 1
add_initial_level()

tab1_content = widgets.VBox([
    widgets.HTML("<p style='color:#666;'><i>Create new hierarchy from scratch</i></p>"),
    hierarchy_type_input,
    widgets.HTML("<br><p style='font-weight:bold;'>🏛️ Boundary Levels:</p>"),
    levels_container,
    add_level_btn,
    widgets.HTML("<br>"),
    create_hierarchy_btn,
    generate_template_btn,
    download_template_btn,
    widgets.HTML("<br>"),
    create_output
])

# Build Tab 2
tab2_content = widgets.VBox([
    widgets.HTML("<p style='color:#666;'><i>Use existing hierarchy</i></p>"),
    search_btn,
    widgets.HTML("<br>"),
    hierarchy_dropdown,
    search_generate_btn,
    search_download_btn,
    widgets.HTML("<br>"),
    search_output
])

tab_selector.children = [tab1_content, tab2_content]
tab_selector.set_title(0, '🏗️ Create New')
tab_selector.set_title(1, '🔍 Use Existing')

main_ui = widgets.VBox([
    widgets.HTML("<h3>🔍 Step 2.1: Boundary Hierarchy & Template</h3>"),
    widgets.HTML("<h4 style='color:red;'><strong>Note:</strong> Boundary data seeding must be performed at the city level. Please authenticate using city-level credentials before proceeding.click below button to scroll to login screen</h4>"),
     boundary_login_again_button,
    tenant_input,
    widgets.HTML("<br>"),
    tab_selector
])

display(main_ui)

## 📤 Step 2.2: Upload Boundary Data

After generating the boundary template:
1. Fill in the boundary hierarchy (State → District → City → Ward)
2. Upload the filled template using the widget below
3. System will create boundary entities with parent-child relationships

**⏱️ Time:** Upload may take 2-5 minutes depending on boundary count.

---

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from unified_loader import APIUploader
import os
import time

print("="*70)
print("   📤 PHASE 2.2: UPLOAD & PROCESS BOUNDARY DATA")
print("="*70)

UPLOADED_FILESTORE_ID = None
PROCESSED_FILESTORE_ID = None
PROCESSED_DOWNLOAD_URL = None


if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

uploader = UPLOADER

boundary_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

upload_process_btn = widgets.Button(
    description='📤 Upload & Process',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

check_status_btn = widgets.Button(
    description='🔍 Check Processing Status',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

download_processed_btn = widgets.Button(
    description='📥 Download Processed File',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

output = widgets.Output()

def on_upload_process_clicked(b):
    global UPLOADED_FILESTORE_ID, PROCESSED_FILESTORE_ID, PROCESSED_DOWNLOAD_URL

    with output:
        clear_output()

        if not BOUNDARY_TENANT:
            print("❌ Complete Step 2.1 first")
            return

        if not BOUNDARY_HIERARCHY_TYPE:
            print("❌ No hierarchy type selected")
            return

        if not boundary_file_upload.value:
            print("❌ Select filled template")
            return

        os.makedirs('upload', exist_ok=True)
        uploaded_file = boundary_file_upload.value[0]
        content = uploaded_file['content']
        filename = f'boundary_filled_{BOUNDARY_TENANT}_{BOUNDARY_HIERARCHY_TYPE}.xlsx'

        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)

        print(f"📄 Saved: {filename}")

        print(f"\n📤 Uploading to filestore...")
        filestore_id = uploader.upload_file_to_filestore(
            file_path=upload_path,
            tenant_id=BOUNDARY_TENANT,
            module='HCM-ADMIN-CONSOLE'
        )

        if not filestore_id:
            print("❌ Upload failed")
            return

        UPLOADED_FILESTORE_ID = filestore_id

        print(f"\n⚙️ Processing...")
        result = uploader.process_boundary_data(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=filestore_id,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            action='create'
        )

        if result:
            status = result.get('status')
            processed_id = result.get('processedFileStoreId')

            print(f"\n✅ Submitted")
            print(f"   Status: {status}")

            if status == 'data-accepted':
                print("\n" + "="*70)
                print("🎉 PHASE 2 COMPLETED!")
                print("="*70)
                print("✅ Boundaries created")
                print("\n⏳ Processing in background...")
                print("   Click 'Check Status' after 10-20 seconds")
                print("="*70)
                check_status_btn.layout.visibility = 'visible'
            elif processed_id:
                PROCESSED_FILESTORE_ID = processed_id
                download_processed_btn.layout.visibility = 'visible'
                print("\n💡 Processed file ready for download")

upload_process_btn.on_click(on_upload_process_clicked)

def on_check_status_clicked(b):
    global PROCESSED_FILESTORE_ID, PROCESSED_DOWNLOAD_URL
    
    with output:
        print("\n🔍 Checking processing status...")
        
        # Poll the _process endpoint to get status
        boundary_mgmt_url = uploader.boundary_mgmt_url
        url = f"{boundary_mgmt_url}/v1/_process-search"
        
        user_info_copy = uploader.user_info.copy()
        user_info_copy['tenantId'] = BOUNDARY_TENANT
        
        payload = {
            "RequestInfo": {
                "apiId": "Rainmaker",
                "authToken": uploader.auth_token,
                "userInfo": user_info_copy,
                "msgId": f"{int(time.time() * 1000)}|en_IN",
                "plainAccessRequest": {}
            },
            "SearchCriteria": {
                "tenantId": BOUNDARY_TENANT,
                "hierarchyType": BOUNDARY_HIERARCHY_TYPE
            }
        }
        
        try:
            import requests
            response = requests.post(url, json=payload, headers={'Content-Type': 'application/json'})
            response.raise_for_status()
            data = response.json()
            
            resources = data.get('ResourceDetails', [])
            if resources:
                # Sort by creation time to get the LATEST upload (most recent)
                sorted_resources = sorted(
                    resources,
                    key=lambda x: x.get('auditDetails', {}).get('createdTime', 0),
                    reverse=True
                )
                latest = sorted_resources[0]
                status = latest.get('status')
                
                print(f"   📊 Total records: {len(resources)}, Using most recent (created: {latest.get('auditDetails', {}).get('createdTime')})")
                
                # Debug: show all available fields
                print(f"   Status: {status}")
                print(f"   Available fields: {list(latest.keys())}")
                
                # Try different field name variations
                processed_id = (latest.get('processedFileStoreId') or 
                               latest.get('processedFilestoreId') or 
                               latest.get('processedFilestoreid'))
                
                # Parse error details from additionalDetails
                additional_details = latest.get('additionalDetails', {})
                error_detail = ""
                error_code = ""
                
                if isinstance(additional_details, dict) and 'error' in additional_details:
                    try:
                        # Error is a JSON string, parse it
                        error_json = json.loads(additional_details['error'])
                        error_code = error_json.get('code', '')
                        error_detail = error_json.get('description', error_json.get('message', ''))
                    except:
                        error_detail = str(additional_details.get('error', ''))
                
                if processed_id:
                    PROCESSED_FILESTORE_ID = processed_id
                    print(f"   Processed File ID: {processed_id}")
                    
                    # Interpret the status
                    if status == 'failed':
                        error_msg = error_detail or str(additional_details)
                        
                        if 'already present' in error_msg.lower() or 'duplicate' in error_msg.lower():
                            print(f"\n💡 Boundaries already exist - this is expected when re-uploading.")
                            status_color = '#fff3cd'
                            status_text_color = '#856404'
                            status_icon = '⚠️'
                            status_title = 'Boundaries Already Exist'
                            status_msg = 'Some or all boundaries already exist. Download file to see details.'
                        else:
                            print(f"\n❌ Processing failed: {error_msg[:150]}")
                            status_color = '#f8d7da'
                            status_text_color = '#721c24'
                            status_icon = '❌'
                            status_title = 'Processing Failed'
                            status_msg = 'Errors occurred. Download file to see details.'
                    else:
                        status_color = '#d4edda'
                        status_text_color = '#155724'
                        status_icon = '✅'
                        status_title = 'Processed File Ready!'
                        status_msg = 'The boundary data has been processed successfully.'
                    
                    # Get download URL
                    filestore_url = uploader.filestore_url
                    file_url = f"{filestore_url}/v1/files/url"
                    params = {"tenantId": BOUNDARY_TENANT, "fileStoreIds": processed_id}
                    
                    file_response = requests.get(file_url, params=params)
                    file_response.raise_for_status()
                    file_data = file_response.json()
                    
                    file_urls = file_data.get('fileStoreIds', [])
                    if file_urls:
                        PROCESSED_DOWNLOAD_URL = file_urls[0].get('url')
                        
                        download_link_html = f"""
                        <div style='background:{status_color}; padding:15px; border-radius:8px; margin:10px 0;'>
                            <h3 style='margin-top:0; color:{status_text_color};'>{status_icon} {status_title}</h3>
                            <p>{status_msg}</p>
                            <a href='{PROCESSED_DOWNLOAD_URL}' download='boundary_processed.xlsx' 
                               style='display:inline-block; padding:12px 24px; background:#28a745; color:white; 
                                      text-decoration:none; border-radius:6px; font-weight:bold;'>
                                📥 Download Processed File
                            </a>
                        </div>
                        """
                        display(HTML(download_link_html))
                        download_processed_btn.layout.visibility = 'visible'
                        check_status_btn.layout.visibility = 'hidden'
                elif status == 'inprogress':
                    print("\n⏳ Still processing... Check again in 10 seconds")
                else:
                    # No processed file available (failed without processedFilestoreId)
                    print(f"\n❌ Processing failed - no processed file generated")
                    if error_detail:
                        print(f"\n📋 Error Code: {error_code}")
                        print(f"📝 Error Details: {error_detail}")
                    else:
                        print(f"\n📝 Raw error: {str(additional_details)[:300]}")
            else:
                print("   No processing records found")
                
        except Exception as e:
            print(f"❌ Error: {str(e)[:200]}")

check_status_btn.on_click(on_check_status_clicked)

def on_download_processed_clicked(b):
    with output:
        if not PROCESSED_DOWNLOAD_URL:
            print("⚠️ No processed file URL available")
            return
        
        print(f"\n📥 Processed file URL ready!")
        print("   Click the green button above to download")

download_processed_btn.on_click(on_download_processed_clicked)

ui = widgets.VBox([
    widgets.HTML("<h3>📤 Step 2.2: Upload & Process</h3>"),
    widgets.HTML(f"<p style='background:#e7f3ff;padding:10px;border-radius:5px;'>"
                 f"<b>📋 Tenant:</b> {BOUNDARY_TENANT if BOUNDARY_TENANT else 'Not set'}<br>"
                 f"<b>🏛️ Hierarchy:</b> {BOUNDARY_HIERARCHY_TYPE if BOUNDARY_HIERARCHY_TYPE else 'Not set'}</p>"),
    widgets.HTML("<br><p><b>Select Filled Template:</b></p>"),
    boundary_file_upload,
    widgets.HTML("<p style='font-size:11px;color:#ff6600;'><i>⚠️ Make sure template is filled</i></p>"),
    widgets.HTML("<br>"),
    upload_process_btn,
    widgets.HTML("<br>"),
    check_status_btn,
    download_processed_btn,
    widgets.HTML("<br>"),
    output
])

display(ui)

---

# 🟪 PHASE 3: COMMON MASTER DATA

## 📋 Overview
In Phase 3, you will configure all common master data and CRS complaint configurations for your tenant.

**🔐 Authentication Level:** Use **STATE-level user** (e.g., authenticated with `statea`)

## 🎯 What You'll Do:
1. **Enter Target Tenant ID** (e.g., `statea` for state-level or `statea.citya` for city-level tenant)
2. **Upload Common Master Excel** with departments, designations, and complaint types
3. **System automatically creates:**
   - Department records in MDMS
   - Designation records in MDMS
   - Complaint types and sub-types
   - SLA configurations
   - Localization entries for all master data

## 📄 Template Required:
- **Common and Complaint Master.xlsx**

## 📊 Sheets in Template:
1. **Department Master** - List all departments (e.g., Public Works, Sanitation)
2. **Designation Master** - List all designations (e.g., Engineer, Supervisor)
3. **Complaint Type Master** - Main complaint categories
4. **Complaint SubType Master** - Sub-categories under each type

## ✅ What Gets Created:
- Departments with auto-generated codes (e.g., `DEPT_1`, `DEPT_2`)
- Designations with auto-generated codes (e.g., `DESIG_01`, `DESIG_02`)
- Complaint types with IDs and SLA hours
- Complaint sub-types linked to parent types
- Localization messages for all entries (en_IN, hi_IN)

## ⚠️ Important:
- **Use STATE-level authentication** (e.g., `statea`) for this phase
- Complete Phase 1 and Phase 2 before Phase 3
- You can upload to any tenant ID (state or city level)
- Departments and Designations created here will be used in Phase 4 (Employee Creation)

---

## 📤 Step 3.1: Upload Common Master Excel

**Instructions:**
1. Enter the **Target Tenant ID** (e.g., `statea,pg`) in the text box
2. Click **Upload** button
3. Select **Common and Complaint Master.xlsx**
4. Click **Set Tenant & Upload** button

**What happens:**
- System reads all sheets (Departments, Designations, Complaint Types, Sub-Types)
- Validates data format
- Stores file for processing in next steps



In [ ]:
print("="*70)
print("        🎯  ENTER TARGET TENANT & UPLOAD COMMON MASTER")
print("="*70)


# Tenant input widget
tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    placeholder='e.g., pg, pg.citya, pg.cityb',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
common_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined upload button
upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Common Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)
boundary_login_again_button = widgets.HTML(
    """
    <div style="margin-top:12px;">
        <a href="#login-step" style="text-decoration:none;">
            <button style="
                padding:8px 16px;
                font-size:14px;
                background:#f0ad4e;
                border:none;
                border-radius:4px;
                cursor:pointer;">
                🔐 Login Again (State Level)
            </button>
        </a>
    </div>
    """)
boundary_login_again_button.layout.visibility = "visible"

upload_output = widgets.Output()

def on_upload(b):
    global COMMON_MASTER_FILE, SELECTED_TENANT
    global result_dept, result_desig, result_ct
    
    with upload_output:
        clear_output()
        
        # Validate tenant input
        if not tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not common_file_upload.value:
            print("❌ Please select Common Master Excel file")
            return
        
        # Validate authentication
        if not UPLOADER or not UPLOADER.authenticated:
            print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
            return
        
        # Set tenant
        SELECTED_TENANT = tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = common_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Common_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        validation_passed = False
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator(
                base_url=UPLOADER.base_url,
                auth_token=UPLOADER.auth_token,
                user_info=UPLOADER.user_info
            )
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=SELECTED_TENANT,
                schema_code='common.masterschemavalidation'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                validation_passed = True
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                COMMON_MASTER_FILE = None
                SELECTED_TENANT = None
                return
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            validation_passed = True
        
        if not validation_passed:
            return
        
        COMMON_MASTER_FILE = upload_path
        
        print("\n" + "="*70)
        print("  ✅ FILE UPLOADED SUCCESSFULLY")
        print("="*70)
        print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
        print(f"\n📄 File: {upload_path}")
        
        # ============================================================
        # INTEGRATED API CALLS - Start processing immediately
        # ============================================================
        
        print("\n" + "="*70)
        print("[PHASE 3] LOADING DEPARTMENTS & DESIGNATIONS")
        print("="*70)
        
        try:
            # Initialize reader
            reader = UnifiedExcelReader(COMMON_MASTER_FILE)
            uploader = UPLOADER
            
            # Read departments and designations (combined sheet)
            departments, designations, dept_loc, desig_loc, dept_name_to_code = reader.read_departments_designations(SELECTED_TENANT, uploader)
            
            # Create reverse mapping (code to name) for error Excel
            dept_code_to_name = {code: name for name, code in dept_name_to_code.items()}
            
            print(f"\n[INFO] Loaded {len(departments)} department(s)")
            for dept in departments:
                print(f"   - {dept['code']}: {dept['name']}")
            
            print(f"\n[INFO] Loaded {len(designations)} designation(s)")
            for desig in designations[:5]:
                print(f"   - {desig['code']}: {desig['name']} [{desig['department']}]")
            if len(designations) > 5:
                print(f"   ... and {len(designations) - 5} more")
            
            print(f"\n[INFO] Auto-generated {len(dept_loc) + len(desig_loc)} localization entries")
            print(f"\n[INFO] Created department name-to-code mapping: {dept_name_to_code}")
            
            # Store reverse mapping for error Excel
            uploader._dept_code_to_name = dept_code_to_name
            
            # Upload Departments
            result_dept = uploader.create_mdms_data(
                schema_code='common-masters.Department',
                data_list=clean_nans(departments),
                tenant=SELECTED_TENANT,
                sheet_name='Department_And_Designation'
            )
            
            # Upload Designations
            result_desig = uploader.create_mdms_data(
                schema_code='common-masters.Designation',
                data_list=clean_nans(designations),
                tenant=SELECTED_TENANT,
                sheet_name='Department And Desgination Mast',
                excel_file=COMMON_MASTER_FILE
            )
            
            # Upload Localization
            all_loc = dept_loc + desig_loc
            result_loc = uploader.create_localization_messages(
                localization_list=clean_nans(all_loc),
                tenant=SELECTED_TENANT,
                sheet_name='Dept_Desig_Localization'
            )
            
            if result_dept['failed'] == 0 and result_desig['failed'] == 0:
                print("\n✅ [SUCCESS] Departments & Designations uploaded successfully!")
            else:
                print("\n⚠️  [WARNING] Some items failed. Check errors/ folder.")
            
            # ============================================================
            # PHASE 3.2: Complaint Types
            # ============================================================
            
            print("\n" + "="*70)
            print("[PHASE 3] LOADING COMPLAINT TYPES")
            print("="*70)
            
            # Read complaint types - pass dept_name_to_code mapping
            complaint_types, ct_loc = reader.read_complaint_types(SELECTED_TENANT, dept_name_to_code)
            
            print(f"\n[INFO] Loaded {len(complaint_types)} complaint type(s)")
            for ct in complaint_types[:10]:
                dept_info = f" [Dept: {ct.get('department', 'N/A')}]" if ct.get('department') else ""
                print(f"   - {ct['serviceCode']}: {ct['name']}{dept_info}")
            if len(complaint_types) > 10:
                print(f"   ... and {len(complaint_types) - 10} more")
            
            print(f"\n[INFO] Auto-generated {len(ct_loc)} localization entries")
            
            # Upload Complaint Types
            uploader._dept_code_to_name = dept_code_to_name
            result_ct = uploader.create_mdms_data(
                schema_code='RAINMAKER-PGR.ServiceDefs',
                data_list=clean_nans(complaint_types),
                tenant=SELECTED_TENANT,
                sheet_name='Complaint Type Master',
                excel_file=COMMON_MASTER_FILE
            )
            
            # Upload Localization
            result_loc = uploader.create_localization_messages(
                localization_list=clean_nans(ct_loc),
                tenant=SELECTED_TENANT,
                sheet_name='ComplaintTypes_Localization'
            )
            
            if result_ct['failed'] == 0:
                print("\n✅ [SUCCESS] Complaint Types uploaded successfully!")
            else:
                print("\n⚠️  [WARNING] Some complaint types failed. Check errors/ folder.")
            
            print("\n" + "="*70)
            print("🎉 PHASE 3 COMPLETED!")
            print("="*70)
            
        except Exception as e:
            print(f"\n❌ ERROR during data processing: {str(e)}")
            import traceback
            print("\nFull traceback:")
            traceback.print_exc()
            print("\n⚠️  Please check the error and try again.")

upload_btn.on_click(on_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
upload_box = widgets.VBox([
    widgets.HTML("<h3>🎯 Step 3.1: Enter Target Tenant & Upload Common Master</h3>"),
       widgets.HTML("<p style='color: #666;'><i>Please log in again using state-level credentials. eg:statea</i></p>"),
    boundary_login_again_button,
    widgets.HTML("<p style='color: #666;'><i>Enter the tenant ID and upload the Common Master Excel file</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 You can upload to any tenant - it doesn't have to be from Phase 1</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Common Master Excel File:</b></p>"),
    common_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated, read, and uploaded to API automatically</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    upload_output
])

display(upload_box)


---

# 📊 INDVIDUAL SUMMARY REPORT

## 📈 Upload Status

This report shows the complete status of all uploaded data across all phases:

**Status Indicators:**
- ✅ **Created:** Successfully uploaded new records to MDMS
- ⚠️ **Already Exists:** Duplicate records (skipped, no action needed)
- ❌ **Failed:** Records with errors (see error messages)

---

## 📥 Download Updated Excel Files

Your original Excel files have been updated with status columns:
- **_STATUS** - SUCCESS / EXISTS / FAILED
- **_STATUS_CODE** - HTTP status code (200, 400, 500, etc.)
- **_ERROR_MESSAGE** - Detailed error description for failed records

**Color Coding:**
- 🟢 **GREEN rows:** Successfully created
- 🟡 **YELLOW rows:** Already exist (duplicates)
- 🔴 **RED rows:** Failed (check _ERROR_MESSAGE)

---

## 🔄 To Retry Failed Records:
1. Download the updated Excel file below
2. Fix the errors based on _ERROR_MESSAGE column
3. Delete the status columns (_STATUS, _STATUS_CODE, _ERROR_MESSAGE)
4. Re-upload the file in the respective phase

---

In [ ]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

# if 'result_tenants' in locals():
#     summary_data.append({
#         'module': 'Tenants',
#         'created': result_tenants.get('created', 0),
#         'exists': result_tenants.get('exists', 0),
#         'failed': result_tenants.get('failed', 0),
#         'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
#         'sheet': 'Tenant Info'
#     })

if 'result_dept' in locals():
    summary_data.append({
        'module': 'Departments',
        'created': result_dept.get('created', 0),
        'exists': result_dept.get('exists', 0),
        'failed': result_dept.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_desig' in locals():
    summary_data.append({
        'module': 'Designations',
        'created': result_desig.get('created', 0),
        'exists': result_desig.get('exists', 0),
        'failed': result_desig.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_ct' in locals():
    summary_data.append({
        'module': 'Complaint Types',
        'created': result_ct.get('created', 0),
        'exists': result_ct.get('exists', 0),
        'failed': result_ct.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Complaint Type Master'
    })

# if 'result_employees' in locals():
#     summary_data.append({
#         'module': 'Employees (HRMS)',
#         'created': result_employees.get('created', 0),
#         'exists': result_employees.get('exists', 0),
#         'failed': result_employees.get('failed', 0),
#         'excel_file': EMPLOYEE_MASTER_FILE if 'EMPLOYEE_MASTER_FILE' in locals() else None,
#         'sheet': 'Employee Master'
#     })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""



if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS/HRMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔒 Note:</b> Status columns are <b>protected</b> and cannot be edited. Fix data in other columns and re-upload if needed.
        </p>
    </div>
    """



html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")

---

# 🟦 PHASE 4: EMPLOYEE BULK CREATION

In Phase 4, you will bulk create employees for your tenant.

**🔐 Authentication Level:** Use **STATE-level user** (e.g., authenticated with `statea`)

## 📋 Overview:
1. **Generate Dynamic Template** - Creates Excel with dropdowns pre-filled from MDMS
2. **Fill Employee Data** - Use dropdowns for departments, designations, roles
3. **Upload & Create** - Bulk create employees via API

## 🎯 What You'll Do:
1. Enter **Target Tenant ID** (e.g., `statea`)
2. Generate dynamic employee template with pre-populated dropdowns
3. Fill in employee information in the Excel template
4. Upload completed template to create employees

## ✅ What Gets Created:
- Employee records with auto-generated employee codes
- User accounts with assigned roles
- Employee-to-boundary assignments
- Localization entries for employee data

## ⚠️ Important:
- **Use STATE-level authentication** (e.g., `statea`) for this phase
- Complete Phase 1, 2, and 3 before Phase 4
- Departments and Designations from Phase 3 will appear in template dropdowns
- Boundaries from Phase 2 will be available for assignment

---

---

## 🎨 Step 4.1: Generate Dynamic Employee Template

**Instructions:**
1. Enter the **Target Tenant ID** (e.g., `pg.citya`) in the text box
2. Click **🎨 Generate Dynamic Template** button
3. Wait for template generation (may take 10-20 seconds)

**What happens:**
- System fetches **Departments** from MDMS → Creates dropdown
- System fetches **Designations** from MDMS → Creates dropdown
- System fetches **Roles** from MDMS → Creates README sheet
- System fetches **Boundaries** from Boundary Service → Creates dropdown
- Creates **Employee Master** sheet with data validation
- Creates **Instructions** sheet with usage guide
- Creates **README - Roles** sheet with all available roles

**Generated File Location:**
```
templates/Employee_Master_Dynamic_{tenant}.xlsx
```

**Template Contains:**
- ✅ All dropdowns pre-filled from MDMS
- ✅ Excel date pickers for date fields
- ✅ README - Roles sheet with copy-paste ready role names
- ✅ Instructions sheet with complete guidance
- ✅ Sample row with default values

💡 **Tip**: This template is customized for your tenant with real data from MDMS!


In [ ]:
print("="*70)
print("        🎨  GENERATE DYNAMIC EMPLOYEE TEMPLATE FROM MDMS")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

print()

# Tenant input for template generation
template_tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    placeholder='e.g., pg.citya, pb.amritsar',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)
# Generate button
generate_employee_template_btn = widgets.Button(
    description='🎨 Generate Dynamic Template',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

template_gen_output = widgets.Output()

GENERATED_EMP_TEMPLATE_PATH = None

def on_generate_emp_template(b):
    global GENERATED_EMP_TEMPLATE_PATH
    
    with template_gen_output:
        clear_output()
        
        if not template_tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        tenant = template_tenant_input.value.strip().lower()
        
        print(f"🔄 Generating dynamic template for tenant: {tenant}")
        print("   Fetching data from MDMS...\n")
        
        uploader = UPLOADER
        
        try:
            template_path = uploader.generate_employee_template(tenant)
            GENERATED_EMP_TEMPLATE_PATH = template_path
            
            download_link_html = f"""
            <div style='background:#d4edda; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#155724;'>✅ Dynamic Template Generated!</h3>
                <p><b>File:</b> {template_path}</p>
                <a href='{template_path}' download='Employee_Master_Dynamic.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#28a745; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold; margin:10px 0;'>
                    📥 Download Template
                </a>
                <p style='margin-top:15px;'><b>✨ Features:</b></p>
                <ul style='margin:5px 0;'>
                    <li>✅ Departments fetched from MDMS</li>
                    <li>✅ Designations fetched from MDMS</li>
                    <li>✅ Roles fetched from User Service</li>
                    <li>✅ Boundaries fetched from Boundary Service</li>
                    <li>✅ 8 pre-filled dropdowns</li>
                    <li>✅ Employee code auto-generated</li>
                    <li>✅ Sample row included</li>
                </ul>
            </div>
            """
            display(HTML(download_link_html))
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")

generate_employee_template_btn.on_click(on_generate_emp_template)

if 'UPLOADED_TENANTS' in locals() and UPLOADED_TENANTS:
    emp_ref = widgets.HTML(
        f"<p style='background:#f0f0f0; padding:10px; border-radius:5px;'>"
        f"<b>📋 Tenants from Phase 1:</b> {', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    emp_ref = widgets.HTML("")

display(widgets.VBox([
    widgets.HTML("<h3>🎨 Step 4.0: Generate Dynamic Template</h3>"),
    widgets.HTML("<div style='background:#fff3cd;padding:12px;border-radius:5px;'>"
                 "<b>💡 Recommended:</b> Generate template with live MDMS data - dropdowns auto-filled!</div>"),
    emp_ref,
    widgets.HTML("<br>"),
    template_tenant_input,
    widgets.HTML("<br>"),
    generate_employee_template_btn,
    template_gen_output
]))

---

### 📝 NEXT STEPS: Fill the Template

**After generating the template:**

1. **Open the Excel file** from the `templates/` folder
2. **Read the 'Instructions' sheet** for detailed guidance
3. **Check 'README - Roles' sheet** for available role names
4. **Fill the 'Employee Master' sheet** with employee data:
   - Use NAMES for departments, designations
   - Role code  
   - Use Excel date picker for dates
   - For multiple roles: copy from README and join with commas (e.g., `Employee,CRS Viewer,CRS Admin`)
5. **Delete the sample row** before uploading
6. **Save the file** and proceed to Step 4.1 below

💡 **TIP**: The 'README - Roles' sheet has all available roles with descriptions - just copy the role names you need!

In [ ]:
print("="*70)
print("        👥  PHASE 4: UPLOAD EMPLOYEE MASTER")
print("="*70)
print()

# Global variable for employee file
EMPLOYEE_MASTER_FILE = None
EMPLOYEE_TENANT = None

# Tenant input widget
employee_tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    placeholder='e.g., pg.citya, pb.amritsar',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
employee_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Upload button
employee_upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Employee Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

employee_upload_output = widgets.Output()

def on_employee_upload(b):
    global EMPLOYEE_MASTER_FILE, EMPLOYEE_TENANT
    global result_employees
    
    with employee_upload_output:
        clear_output()
        
        # Validate tenant input
        if not employee_tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not employee_file_upload.value:
            print("❌ Please select Employee Master Excel file")
            return
        
        # Validate authentication
        if not UPLOADER or not UPLOADER.authenticated:
            print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
            return
        
        # Set tenant
        EMPLOYEE_TENANT = employee_tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = employee_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Employee_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        EMPLOYEE_MASTER_FILE = upload_path
        
        print("="*70)
        print("  ✅ FILE UPLOADED SUCCESSFULLY")
        print("="*70)
        print(f"\n🎯 Target Tenant: {EMPLOYEE_TENANT}")
        print(f"\n📄 File: {upload_path}")
        
        # ============================================================
        # INTEGRATED API CALLS - Start processing immediately
        # ============================================================
        
        print("\n" + "="*70)
        print("[PHASE 4] BULK CREATING EMPLOYEES")
        print("="*70)
        
        try:
            # Initialize reader and uploader
            reader = UnifiedExcelReader(EMPLOYEE_MASTER_FILE)
            uploader = UPLOADER
            
            # Read employees using bulk method
            employees = reader.read_employees_bulk(EMPLOYEE_TENANT, UPLOADER)
            
            print(f"\n[INFO] Loaded {len(employees)} employee(s) from Excel")
            for emp in employees[:5]:
                dept = emp['assignments'][0]['department']
                desig = emp['assignments'][0]['designation']
                roles = [r['code'] for r in emp['user']['roles']]
                print(f"   - {emp['code']}: {emp['user']['name']} [{dept}/{desig}] Roles: {', '.join(roles)}")
            if len(employees) > 5:
                print(f"   ... and {len(employees) - 5} more")
            
            # Create employees
            result_employees = uploader.create_employees(
                employee_list=clean_nans(employees),
                tenant=EMPLOYEE_TENANT,
                sheet_name='Employee Master',
                excel_file=EMPLOYEE_MASTER_FILE
            )
            
            # Summary
            if result_employees['failed'] == 0:
                print("\n✅ [SUCCESS] Employees created successfully!")
                print("\n🎉 PHASE 4 COMPLETED!")
            else:
                print("\n⚠️  [WARNING] Some employees failed. Check errors/ folder or status columns in Excel.")
            
        except Exception as e:
            print(f"\n❌ ERROR during employee creation: {str(e)}")
            import traceback
            print("\nFull traceback:")
            traceback.print_exc()
            print("\n⚠️  Please check the error and try again.")

employee_upload_btn.on_click(on_employee_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
employee_upload_box = widgets.VBox([
    widgets.HTML("<h3>👥 Step 4.1: Upload Employee Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Upload Employee Master Excel to bulk create employees</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    employee_tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 Employees will be created for this tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Employee Master Excel File:</b></p>"),
    employee_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be read and employees created automatically</i></p>"),
    widgets.HTML("<br>"),
    employee_upload_btn,
    employee_upload_output
])

display(employee_upload_box)


---
# 📊 INDVIDUAL SUMMARY REPORT
## 📈 Upload Status

This report shows the complete status of all uploaded data across all phases:

**Status Indicators:**
- ✅ **Created:** Successfully uploaded new records to MDMS
- ⚠️ **Already Exists:** Duplicate records (skipped, no action needed)
- ❌ **Failed:** Records with errors (see error messages)

---

## 📥 Download Updated Excel Files

Your original Excel files have been updated with status columns:
- **_STATUS** - SUCCESS / EXISTS / FAILED
- **_STATUS_CODE** - HTTP status code (200, 400, 500, etc.)
- **_ERROR_MESSAGE** - Detailed error description for failed records

**Color Coding:**
- 🟢 **GREEN rows:** Successfully created
- 🟡 **YELLOW rows:** Already exist (duplicates)
- 🔴 **RED rows:** Failed (check _ERROR_MESSAGE)

---

## 🔄 To Retry Failed Records:
1. Download the updated Excel file below
2. Fix the errors based on _ERROR_MESSAGE column
3. Delete the status columns (_STATUS, _STATUS_CODE, _ERROR_MESSAGE)
4. Re-upload the file in the respective phase

---

In [ ]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []


if 'result_employees' in locals():
    summary_data.append({
        'module': 'Employees (HRMS)',
        'created': result_employees.get('created', 0),
        'exists': result_employees.get('exists', 0),
        'failed': result_employees.get('failed', 0),
        'excel_file': EMPLOYEE_MASTER_FILE if 'EMPLOYEE_MASTER_FILE' in locals() else None,
        'sheet': 'Employee Master'
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""



if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS/HRMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔒 Note:</b> Status columns are <b>protected</b> and cannot be edited. Fix data in other columns and re-upload if needed.
        </p>
    </div>
    """



html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")

---

# 🔵 PHASE 5: WORKFLOW CONFIGURATION MANAGEMENT

In Phase 5, you can download, edit, and upload workflow configurations for your tenant.

**🔐 Authentication Level:** Use **STATE-level user** (e.g., authenticated with `statea`)

## 📋 Overview:
Workflow configurations define the state machine for complaint processing:
- **States**: Different stages of a complaint (PENDING, ASSIGNED, RESOLVED, etc.)
- **Actions**: Transitions between states (ASSIGN, RESOLVE, REJECT, etc.)
- **Roles**: Who can perform each action (GRO, PGR_LME, CITIZEN, etc.)
- **SLA**: Service level agreement timeouts for each state

## 🎯 What You'll Do:
1. **Download existing workflow** configuration as JSON
2. **Edit the JSON file** to customize states, actions, roles, or SLA
3. **Upload the modified JSON** to create or update the workflow

## ✅ What Gets Created/Updated:
- Workflow business service configuration
- State definitions and transitions
- Role-based access control for actions
- SLA timings for each state

## ⚠️ Important:
- System automatically detects if workflow exists (updates) or not (creates)
- Default workflow is **PGR** (Public Grievance Redressal)
- Be careful when editing - invalid JSON will fail
- Always download first to see the current configuration

---


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os

print("="*70)
print("        🔄  PHASE 5: WORKFLOW CONFIGURATION MANAGEMENT")
print("="*70)

# if not UPLOADER or not UPLOADER.authenticated:
#     print("❌ Please authenticate first! Run the Authentication cell (Cell 7).")
#     raise SystemExit("Authentication required")

uploader = UPLOADER

# =============================================================================
# SECTION 5.1: DOWNLOAD WORKFLOW CONFIGURATION
# =============================================================================

download_tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    description='🏛️ Tenant ID:',
    placeholder='e.g., pg, statea',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

download_business_service_input = widgets.Text(
    value='PGR',
    description='📋 Business Service:',
    placeholder='e.g., PGR',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

download_btn = widgets.Button(
    description='📥 Download Workflow JSON',
    button_style='primary',
    icon='download',
    layout=widgets.Layout(width='95%', height='40px')
)

download_output = widgets.Output()

DOWNLOADED_WORKFLOW_PATH = None

def on_download_clicked(b):
    global DOWNLOADED_WORKFLOW_PATH
    
    with download_output:
        clear_output()
        
        tenant = download_tenant_input.value.strip()
        business_service = download_business_service_input.value.strip()
        
        if not tenant or not business_service:
            print("❌ Please enter tenant ID and business service")
            return
        
        try:
            file_path = uploader.download_workflow_json(
                tenant=tenant,
                business_service=business_service
            )
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")

download_btn.on_click(on_download_clicked)

# =============================================================================
# SECTION 5.2: UPLOAD WORKFLOW CONFIGURATION
# =============================================================================

upload_file_widget = widgets.FileUpload(
    accept='.json',
    multiple=False,
    description='📄 Upload JSON:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

upload_btn = widgets.Button(
    description='🚀 Apply Workflow',
    button_style='success',
    icon='upload',
    layout=widgets.Layout(width='95%', height='40px'),
    disabled=True  # Disabled until file is uploaded
)

# Login again button for Phase 5
phase5_login_again_button = widgets.HTML(
    """
    <div style="margin-top:12px;">
        <a href="#login-step" style="text-decoration:none;">
            <button style="
                padding:8px 16px;
                font-size:14px;
                background:#f0ad4e;
                border:none;
                border-radius:4px;
                cursor:pointer;">
                🔐 Login Again
            </button>
        </a>
    </div>
    """)
phase5_login_again_button.layout.visibility = "hidden"

upload_output = widgets.Output()

UPLOADED_WORKFLOW_FILE = None

def on_file_upload(change):
    global UPLOADED_WORKFLOW_FILE
    
    if change['new']:
        # Handle file upload - change['new'] is a tuple in newer ipywidgets
        uploaded_files = change['new']
        
        if isinstance(uploaded_files, tuple) and len(uploaded_files) > 0:
            # Newer ipywidgets: tuple of file info dicts
            uploaded_file = uploaded_files[0]
            filename = uploaded_file['name']
            content = uploaded_file['content']
        elif isinstance(uploaded_files, dict):
            # Older ipywidgets: dict with filename as key
            filename = list(uploaded_files.keys())[0]
            uploaded_file = uploaded_files[filename]
            content = uploaded_file['content']
        else:
            with upload_output:
                clear_output()
                print("❌ Unexpected file upload format")
            return
        
        # Save to temporary location
        temp_path = f"uploaded_workflow_{filename}"
        with open(temp_path, 'wb') as f:
            f.write(content)
        
        UPLOADED_WORKFLOW_FILE = temp_path
        upload_btn.disabled = False
        
        with upload_output:
            clear_output()
            print(f"✅ File uploaded: {filename}")
            print(f"   Saved to: {temp_path}")
            print(f"\n📝 Click 'Apply Workflow' button to upload")

upload_file_widget.observe(on_file_upload, names='value')

def on_upload_clicked(b):
    global UPLOADED_WORKFLOW_FILE
    
    with upload_output:
        clear_output()
        
        if not UPLOADED_WORKFLOW_FILE:
            print("❌ Please upload a JSON file first")
            return
        
        if not os.path.exists(UPLOADED_WORKFLOW_FILE):
            print(f"❌ File not found: {UPLOADED_WORKFLOW_FILE}")
            return
        
        try:
            result = uploader.create_or_update_workflow(UPLOADED_WORKFLOW_FILE)
            
            if result.get('success'):
                print("\n" + "="*70)
                print("🎉 PHASE 5 COMPLETED!")
                phase5_login_again_button.layout.visibility = "visible"
                print("\n💡 Workflow configuration applied successfully. Use 'Login Again' if you need to switch tenant context")
                print("="*70)
            else:
                print(f"\n❌ Upload failed: {result.get('error')}")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

upload_btn.on_click(on_upload_clicked)

# =============================================================================
# UI LAYOUT
# =============================================================================

ui = widgets.VBox([
    widgets.HTML("<h3>🔍 Step 5.1: Download Workflow Configuration</h3>"),
    widgets.HTML("<p style='color:#666;'><i>Download existing workflow to see current configuration</i></p>"),
    download_tenant_input,
    download_business_service_input,
    download_btn,
    download_output,
    
    widgets.HTML("<br><hr><br>"),
    
    widgets.HTML("<h3>🚀 Step 5.2: Upload Modified Workflow</h3>"),
    widgets.HTML("<p style='color:#666;'><i>Upload your edited JSON file (system auto-detects create vs update)</i></p>"),
    upload_file_widget,
    upload_btn,
    upload_output,
    phase5_login_again_button
])

display(ui)


---

# 📊 FINAL SUMMARY REPORT

## 📈 Upload Status

This report shows the complete status of all uploaded data across all phases:

**Status Indicators:**
- ✅ **Created:** Successfully uploaded new records to MDMS
- ⚠️ **Already Exists:** Duplicate records (skipped, no action needed)
- ❌ **Failed:** Records with errors (see error messages)

---

## 📥 Download Updated Excel Files

Your original Excel files have been updated with status columns:
- **_STATUS** - SUCCESS / EXISTS / FAILED
- **_STATUS_CODE** - HTTP status code (200, 400, 500, etc.)
- **_ERROR_MESSAGE** - Detailed error description for failed records

**Color Coding:**
- 🟢 **GREEN rows:** Successfully created
- 🟡 **YELLOW rows:** Already exist (duplicates)
- 🔴 **RED rows:** Failed (check _ERROR_MESSAGE)

---

## 🔄 To Retry Failed Records:
1. Download the updated Excel file below
2. Fix the errors based on _ERROR_MESSAGE column
3. Delete the status columns (_STATUS, _STATUS_CODE, _ERROR_MESSAGE)
4. Re-upload the file in the respective phase

---

In [ ]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

if 'result_tenants' in locals():
    summary_data.append({
        'module': 'Tenants',
        'created': result_tenants.get('created', 0),
        'exists': result_tenants.get('exists', 0),
        'failed': result_tenants.get('failed', 0),
        'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
        'sheet': 'Tenant Info'
    })

if 'result_dept' in locals():
    summary_data.append({
        'module': 'Departments',
        'created': result_dept.get('created', 0),
        'exists': result_dept.get('exists', 0),
        'failed': result_dept.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_desig' in locals():
    summary_data.append({
        'module': 'Designations',
        'created': result_desig.get('created', 0),
        'exists': result_desig.get('exists', 0),
        'failed': result_desig.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_ct' in locals():
    summary_data.append({
        'module': 'Complaint Types',
        'created': result_ct.get('created', 0),
        'exists': result_ct.get('exists', 0),
        'failed': result_ct.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Complaint Type Master'
    })

if 'result_employees' in locals():
    summary_data.append({
        'module': 'Employees (HRMS)',
        'created': result_employees.get('created', 0),
        'exists': result_employees.get('exists', 0),
        'failed': result_employees.get('failed', 0),
        'excel_file': EMPLOYEE_MASTER_FILE if 'EMPLOYEE_MASTER_FILE' in locals() else None,
        'sheet': 'Employee Master'
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""



if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS/HRMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔒 Note:</b> Status columns are <b>protected</b> and cannot be edited. Fix data in other columns and re-upload if needed.
        </p>
    </div>
    """



html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")